# HYPERPARAMTER TUNING USING TENSORBOARD

In [1]:
# Import Depedencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorboard import notebook
import datetime
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
from tensorboard.plugins.hparams import api as hp
import random

In [2]:
# Load Malaria Dataset From Tensorflow Datasets
dataset, datasetInfo = tfds.load("malaria", 
                                 with_info=True, 
                                 shuffle_files=True) # -----------++++++    ->    ---+++-++--+++-

# Dataset Split Function
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    trainDataset = dataset.take(int(TRAIN_RATIO*len(dataset)))
    leftDataset = dataset.skip(int(TRAIN_RATIO*len(dataset)))
    valDataset = leftDataset.take(int(VAL_RATIO*len(dataset)))
    testDataset = leftDataset.skip(int(VAL_RATIO*len(dataset)))
    return trainDataset, valDataset, testDataset

# Split Dataset As 80/10/10 
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
trainDataset, valDataset, testDataset = split(dataset['train'], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IMAGE_SIZE = 224
def resizeRescale(inputs):
    return tf.image.resize(inputs['image'], (IMAGE_SIZE, IMAGE_SIZE))/255., inputs['label']
trainDataset = trainDataset.map(resizeRescale)
valDataset = valDataset.map(resizeRescale)
testDataset = testDataset.map(resizeRescale)

# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = trainDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-26 09:47:10.419288: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-26 09:47:10.419315: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-26 09:47:10.419318: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-26 09:47:10.419334: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-26 09:47:10.419345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## MODEL TUNE FUNCTION

In [3]:
def modelTune(hparams):
    # Using Sequential API with Hyperparamters
    IMAGE_SIZE = 224
    model = tf.keras.Sequential([
        Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

        Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
        BatchNormalization(),
        MaxPooling2D(pool_size=2, strides=2),
        Dropout(rate = hparams[HP_DROPOUT_RATE]),

        Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
        BatchNormalization(),
        MaxPooling2D(pool_size=2, strides=2),

        Flatten(),

        Dense(hparams[HP_NUM_UNITS_1], activation='relu', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
        BatchNormalization(),
        Dropout(rate = hparams[HP_DROPOUT_RATE]),

        Dense(hparams[HP_NUM_UNITS_2], activation='relu', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
        BatchNormalization(),
        Dropout(rate = hparams[HP_DROPOUT_RATE]),

        Dense(1, activation='sigmoid'),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE]),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(trainDataset, epochs=1)
    _, accuracy = model.evaluate(valDataset)
    return accuracy



## HYPER-PARAMETERS RANGE

In [4]:
HP_NUM_UNITS_1 = hp.HParam('NUM_UNITS_1', hp.Discrete([16, 32, 64, 128]))
HP_NUM_UNITS_2 = hp.HParam('NUM_UNITS_2', hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT_RATE = hp.HParam('DROPOUT_RATE', hp.Discrete([0.1, 0.2, 0.3]))
HP_REGULARIZATION_RATE = hp.HParam('REGULARIZATION', hp.Discrete([0.001, 0.01, 0.1]))
HP_LEARNING_RATE = hp.HParam('LEARNING_RATE', hp.Discrete([1e-4, 1e-3]))

## GRID SEARCH METHOD TO TUNE
### -> Very slow but better results

In [ ]:
runNumber = 0
for NUM_UNITS_1 in HP_NUM_UNITS_1.domain.values:
    for NUM_UNITS_2 in HP_NUM_UNITS_2.domain.values:
        for DROPOUT_RATE in HP_DROPOUT_RATE.domain.values:
            for REGULARIZATION_RATE in HP_REGULARIZATION_RATE.domain.values:
                for LEARNING_RATE in HP_LEARNING_RATE.domain.values:
                    hparams = {
                        HP_NUM_UNITS_1 : NUM_UNITS_1,
                        HP_NUM_UNITS_2 : NUM_UNITS_2,
                        HP_DROPOUT_RATE : DROPOUT_RATE,
                        HP_REGULARIZATION_RATE : REGULARIZATION_RATE,
                        HP_LEARNING_RATE : LEARNING_RATE
                    }

                    fileWriter = tf.summary.create_file_writer('logs/HParamsTuning/' + str(runNumber+1))

                    with fileWriter.as_default():
                        hp.hparams(hparams)
                        accuracy = modelTune(hparams)
                        tf.summary.scalar('accuracy', data = accuracy, step = runNumber+1)
                    print("For Run : {}, Our Hyper-Parameters are : \n NUM_UNITS_1 : {}\n NUM_UNITS_2 : {}\n DROPOUT_RATE : {}\n REGULARIZATION_RATE : {}\n LEARNING_RATE : {}\n".format(runNumber+1, hparams[HP_NUM_UNITS_1], hparams[HP_NUM_UNITS_2], hparams[HP_DROPOUT_RATE], hparams[HP_REGULARIZATION_RATE], hparams[HP_LEARNING_RATE]))
                    runNumber+=1

## RANDOM SEARCH METHOD TO TUNE
### -> Faster but not that good results

In [5]:
# Convert hyperparameter ranges to lists
NUM_UNITS_1_list = HP_NUM_UNITS_1.domain.values
NUM_UNITS_2_list = HP_NUM_UNITS_2.domain.values
DROPOUT_RATE_list = HP_DROPOUT_RATE.domain.values
REGULARIZATION_RATE_list = HP_REGULARIZATION_RATE.domain.values
LEARNING_RATE_list = HP_LEARNING_RATE.domain.values

nRandomSearches = 100
# Perform random search
runNumber = 0
for _ in range(nRandomSearches):
    # Randomly sample hyperparameters
    hparams = {
        HP_NUM_UNITS_1: random.choice(NUM_UNITS_1_list),
        HP_NUM_UNITS_2: random.choice(NUM_UNITS_2_list),
        HP_DROPOUT_RATE: random.choice(DROPOUT_RATE_list),
        HP_REGULARIZATION_RATE: random.choice(REGULARIZATION_RATE_list),
        HP_LEARNING_RATE: random.choice(LEARNING_RATE_list)
    }

    fileWriter = tf.summary.create_file_writer(f'logs/HParamsTuning/{runNumber+1}')

    with fileWriter.as_default():
        hp.hparams(hparams)
        accuracy = modelTune(hparams)
        tf.summary.scalar('accuracy', data=accuracy, step=runNumber+1)

    print(f"Run {runNumber+1}: Hyperparameters are:")
    print(f"  NUM_UNITS_1: {hparams[HP_NUM_UNITS_1]}")
    print(f"  NUM_UNITS_2: {hparams[HP_NUM_UNITS_2]}")
    print(f"  DROPOUT_RATE: {hparams[HP_DROPOUT_RATE]}")
    print(f"  REGULARIZATION_RATE: {hparams[HP_REGULARIZATION_RATE]}")
    print(f"  LEARNING_RATE: {hparams[HP_LEARNING_RATE]}\n")

    runNumber += 1

2025-01-26 09:47:16.539761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 53ms/step - accuracy: 0.5500 - loss: 1.3407
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5531 - loss: 1.0947
Run 1: Hyperparameters are:
  NUM_UNITS_1: 16
  NUM_UNITS_2: 16
  DROPOUT_RATE: 0.3
  REGULARIZATION_RATE: 0.01
  LEARNING_RATE: 0.0001

689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 54ms/step - accuracy: 0.5662 - loss: 0.9026
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6375 - loss: 0.7005
Run 2: Hyperparameters are:
  NUM_UNITS_1: 16
  NUM_UNITS_2: 32
  DROPOUT_RATE: 0.3
  REGULARIZATION_RATE: 0.001
  LEARNING_RATE: 0.0001

689/689 ━━━━━━━━━━━━━━━━━━━━ 38s 53ms/step - accuracy: 0.5618 - loss: 1.2651
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6435 - loss: 0.9515
Run 3: Hyperparameters are:
  NUM_UNITS_1: 16
  NUM_UNITS_2: 16
  DROPOUT_RATE: 0.3
  REGULARIZATION_RATE: 0.01
  LEARNING_RATE: 0.0001

689/689 ━━━━━━━━━━━━━━━━━━━━ 38s 52ms/step - accuracy: 0.5923 - loss: 2.4226
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4980

In [6]:
notebook.start("--logdir logs")

Reusing TensorBoard on port 6007 (pid 38203), started 15:17:02 ago. (Use '!kill 38203' to kill it.)